In [13]:
!pip install langgraph

A simple greeting Node

In [14]:
from typing import Dict, TypedDict, List
from langgraph.graph import StateGraph

In [15]:
class AgentState(TypedDict):
  message: str

def greeting_node(state: AgentState) -> AgentState:
  state['message']="hey " + state["message"]+", how are you?"
  return state

In [16]:
graph=StateGraph(AgentState)
graph.add_node("greeting",greeting_node)
graph.set_entry_point("greeting")
graph.set_finish_point("greeting")

app=graph.compile()

result=app.invoke({"message":"bob"})

result["message"]

'hey bob, how are you?'

Graph 2 => multiple Inputs graph

In [30]:
class AgentStatee(TypedDict):
    values: List[int]
    name: str
    result: str

def process_values(state: AgentStatee) -> AgentStatee:
    state["result"] = "Hi there " + state["name"] + ", Your List Sum is " + str(sum(state["values"]))
    return state

graph = StateGraph(AgentStatee)
graph.add_node("process_values", process_values)
graph.set_entry_point("process_values")
graph.set_finish_point("process_values")

app = graph.compile()
answer = app.invoke({"values": [1, 2, 3, 4, 5], "name": "Bob"})
print(answer)

{'values': [1, 2, 3, 4, 5], 'name': 'Bob', 'result': 'Hi there Bob, Your List Sum is 15'}


In [37]:
class AgentStatee(TypedDict):
    values: List[int]
    name: str
    operation: str
    result: str

def difference(values):
    result = values[0]
    for val in values[1:]:
        result -= val
    return result


def process_values(state: AgentStatee) -> AgentStatee:
    if(state["operation"]=="sum"):
      state["result"] = "Hi there " + state["name"] + ", Your List Sum is " + str(sum(state["values"]))
    #we can also add for multiply, division
    else:
      state["result"] = "Hi there " + state["name"] + ", Your List Sum is " + str(difference(state["values"]))
    return state

graph = StateGraph(AgentStatee)
graph.add_node("process_values", process_values)
graph.set_entry_point("process_values")
graph.set_finish_point("process_values")

app = graph.compile()
answer = app.invoke({"values": [1, 2, 3, 4, 5], "name": "Bob", "operation":"diff"})
print(answer)

{'values': [1, 2, 3, 4, 5], 'name': 'Bob', 'operation': 'diff', 'result': 'Hi there Bob, Your List Sum is -13'}


Sequential Graph

In [40]:
class AgentStatee(TypedDict):
    name: str
    age: int
    final: str

def first_node(state: AgentStatee) -> AgentStatee:
  state["final"] = f"Hi there {state['name']},"
  return state
def second_node(state:AgentStatee)->AgentStatee:
  state["final"] += f"You are {state['age']} years old"
  return state

graph = StateGraph(AgentStatee)
graph.add_node("first_node", first_node)
graph.add_node("second_node", second_node)
graph.set_entry_point("first_node")
graph.add_edge("first_node","second_node")
graph.set_finish_point("second_node")

app = graph.compile()
answer = app.invoke({"name":"bob","age":30})
print(answer)

{'name': 'bob', 'age': '30', 'final': 'Hi there bob,You are 30 years old'}
